# Init

In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [3]:
conn=init_connection(dsn)

2021-07-27 13:38:42.951 INFO    root: Connecting to the PostgreSQL database...
2021-07-27 13:38:42.968 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-07-27 13:38:42.969 INFO    root: PostgreSQL connection status: 0
2021-07-27 13:38:42.970 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


# 23/7/2021

In [17]:
from enum import Enum,IntEnum
from typing import NamedTuple

In [18]:
class Detection(NamedTuple):
    name: str
    prob: float

In [19]:
detection=Detection('car',0.95)

In [35]:
detection.name,detection.prob
detection

Detection(name='car', prob=0.95)

In [37]:
import numpy as np
x=np.array([1,2,3])

In [39]:
np.asarray(x)

array([1, 2, 3])

# 24/7/2021

In [13]:
query_project_SQL="""
                    SELECT
                        p.id,
                            p.name,
                            description,
                            deployment_id,
                            project_path,
                            dt.name as deployment_type
                    FROM
                        public.project p
                        LEFT JOIN deployment_type dt ON dt.id = p.deployment_id
                    WHERE
                        p.id = 2;"""

In [15]:
db_fetchone(query_project_SQL,conn)

Record(id=2, name='First Project', description='This project is for object detection', deployment_id=2, project_path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/project', deployment_type='Object Detection with Bounding Boxes')

### Get column names for DB

In [23]:

data_rows=[]
with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute("""select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;""")
                column_names = [desc[0] for desc in cur.description]
                for row in cur:
                    data_rows.append(row)
print("Column names: {}\n".format(column_names))


Column names: ['ID', 'Name', 'Framework', 'Model Path', 'Date/Time']



In [14]:
from psycopg2.extras import NamedTupleCursor
with conn.cursor(cursor_factory=NamedTupleCursor) as cur:
    cur.execute("""
                select pt.id as "ID", pt.name as "Name",f.name as "Framework",model_path as "Model Path", pt.updated_at as "Date/Time" from pre_trained_models pt left join framework f on framework_id = f.id;
                    """)
    return_all=cur.fetchall()

In [111]:
SQL_msg="""
    select * from json_each_text('{"a":"foo", "b":"bar"}')"""

                    
return_all,column_names=db_fetchall(SQL_msg,conn=conn,fetch_col_name=True)

In [112]:
for i in return_all:
    display((i))

Record(key='a', value='foo')

Record(key='b', value='bar')

In [22]:
column_names=None

In [61]:
import pandas as pd

####  Create Pandas Dataframe using PostgreSQL column names

In [91]:
df=pd.DataFrame(return_all,columns=column_names)
display(df)

,ID,Name,Framework,Deployment Type,Model Path
0,1,[TF] SSD MobileNet V2 FPNLite 320x320,TensorFlow,Object Detection with Bounding Boxes,./pre-trained-models/ssd_mobilenet_v2_fpnlite_...
1,2,[TF] SSD ResNet50 V1 FPN 640x640 (RetinaNet50),TensorFlow,Object Detection with Bounding Boxes,./pre-trained-models/ssd_resnet50_v1_fpn_640x6...


In [92]:
from pathlib import Path
x=Path.home()
x

PosixPath('/home/rchuzh')

In [67]:
y=x/'./programming'

In [68]:
y

PosixPath('/programming')

In [94]:
DATA_DIR = Path.home() / '.local/share/integrated-vision-inspection-system/app_media'
DATA_DIR

PosixPath('/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media')

# 25/7/2021

In [63]:
from core.utils.helper import create_dataframe

In [90]:
df=create_dataframe(return_all,column_names=column_names)
df2=df.loc[(df["Framework"]=="TensorFlow"),"ID":"Framework"]
display(df2.iloc[10:20])

,ID,Name,Framework
No.,,,


In [57]:
x=pd.Series([True],index=["Framework"])
x

Framework    True
dtype: bool

In [97]:
name_list=[PT.Name for PT in return_all]
name_list

['My Second Dataset', 'My Fourth Dataset']

In [28]:
x=[1,2,3,4]
x.insert(0,"Select")
x

['Select', 1, 2, 3, 4]

In [98]:
for dataset in return_all:
    print(list(dataset))

[2, 'My Second Dataset', 4, datetime.datetime(2021, 7, 24, 14, 25, 45, 68588, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))]
[4, 'My Fourth Dataset', 4, datetime.datetime(2021, 7, 24, 14, 25, 45, 68588, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))]


In [113]:
import json

In [114]:
json.dumps(None)

'null'

In [115]:
from psycopg2.extras import Json

In [121]:
print(Json({}))

'{}'


In [4]:
class random:
    def __init__(self,name,age) -> None:
        self.name=name
        self.age=age
    
    @classmethod
    def test(self,string):
        print(string)

In [5]:
test=random("Alex",22)

In [9]:
random.test("test")

test


In [130]:
def transformer(random:random):
    random.name="Mandy"
    

In [131]:
transformer(test)

In [132]:
test.name

'Mandy'

# 26/7/2021

### snippet for Editor

```python
if editor not in session_state:
    session_state.editor=Editor()

deployment_type=selectbox(...)
if session_state.project.deployment_type!=deployment_type:
    session_state.project.editor_config=load_config...
    session_state.editor=Editor(...)
```

In [6]:
query_editor_SQL="""SELECT
    editor_config
FROM
    public.editor
WHERE
    project_id = %s;"""

In [12]:
editor_config=db_fetchone(query_editor_SQL,conn,[7]).editor_config

In [57]:
editor_config="""<View>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true">
        <Label value="Hello" background="#ff9ef4"/>
        <Label value="World" background="#8fd06d"/>
      </RectangleLabels>
    </View>
  </View>
</View>
"""

In [36]:
from xml.dom import minidom

In [37]:
xml_doc=minidom.parseString(editor_config)
xml_doc

In [4]:
from data_export.label_studio_converter.converter import Converter
from tempfile import mkdtemp

In [5]:
# tmp_filename=mkdtemp()
EXPORT_DIR=Path("/home/rchuzh/programming/image_labelling_shrdc/src/test/extra/test").resolve()
# tmp_dir=(EXPORT_DIR/tmp_filename).resolve()
# tmp_dir

In [58]:
from lxml import etree
from collections import defaultdict
import logging
from copy import deepcopy
from enum import Enum
import json
logger = logging.getLogger(__name__)
_LABEL_TAGS = {'Label', 'Choice'}
_NOT_CONTROL_TAGS = {'Filter',}

In [59]:
def parse_config(config_string):
    """
    :param config_string: Label config string
    :return: structured config of the form:
    {
        "<ControlTag>.name": {
            "type": "ControlTag",
            "to_name": ["<ObjectTag1>.name", "<ObjectTag2>.name"],
            "inputs: [
                {"type": "ObjectTag1", "value": "<ObjectTag1>.value"},
                {"type": "ObjectTag2", "value": "<ObjectTag2>.value"}
            ],
            "labels": ["Label1", "Label2", "Label3"] // taken from "alias" if exists or "value"
    }
    """
    if not config_string:
        return {}

    def _is_input_tag(tag):
        return tag.attrib.get('name') and tag.attrib.get('value')

    def _is_output_tag(tag):
        return tag.attrib.get('name') and tag.attrib.get('toName') and tag.tag not in _NOT_CONTROL_TAGS

    def _get_parent_output_tag_name(tag, outputs):
        # Find parental <Choices> tag for nested tags like <Choices><View><View><Choice>...
        parent = tag
        while True:
            parent = parent.getparent()
            if parent is None:
                return
            name = parent.attrib.get('name')
            if name in outputs:
                return name

    xml_tree = etree.fromstring(config_string)

    inputs, outputs, labels = {}, {}, defaultdict(dict)
    for tag in xml_tree.iter():
        if _is_output_tag(tag):
            tag_info = {'type': tag.tag, 'to_name': tag.attrib['toName'].split(',')}
            # Grab conditionals if any
            conditionals = {}
            if tag.attrib.get('perRegion') == 'true':
                if tag.attrib.get('whenTagName'):
                    conditionals = {'type': 'tag', 'name': tag.attrib['whenTagName']}
                elif tag.attrib.get('whenLabelValue'):
                    conditionals = {'type': 'label', 'name': tag.attrib['whenLabelValue']}
                elif tag.attrib.get('whenChoiceValue'):
                    conditionals = {'type': 'choice', 'name': tag.attrib['whenChoiceValue']}
            if conditionals:
                tag_info['conditionals'] = conditionals
            outputs[tag.attrib['name']] = tag_info
        elif _is_input_tag(tag):
            inputs[tag.attrib['name']] = {'type': tag.tag, 'value': tag.attrib['value'].lstrip('$')}
        if tag.tag not in _LABEL_TAGS:
            continue
        parent_name = _get_parent_output_tag_name(tag, outputs)
        if parent_name is not None:
            actual_value = tag.attrib.get('alias') or tag.attrib.get('value')
            if not actual_value:
                logger.debug(
                    'Inspecting tag {tag_name}... found no "value" or "alias" attributes.'.format(
                        tag_name=etree.tostring(tag, encoding='unicode').strip()[:50]))
            else:
                labels[parent_name][actual_value] = dict(tag.attrib)
    for output_tag, tag_info in outputs.items():
        tag_info['inputs'] = []
        for input_tag_name in tag_info['to_name']:
            if input_tag_name not in inputs:
                logger.error(
                    f'to_name={input_tag_name} is specified for output tag name={output_tag}, '
                    'but we can\'t find it among input tags')
                continue
            tag_info['inputs'].append(inputs[input_tag_name])
        tag_info['labels'] = list(labels[output_tag])
        tag_info['labels_attrs'] = labels[output_tag]
    return outputs

In [60]:
editor_config2=parse_config(editor_config)

In [ ]:
editor_config2

In [42]:
def get_export_formats(editor_config):  
    config=parse_config(editor_config)
    converter = Converter(config=config, project_dir=None)
    formats = []
    supported_formats = set(converter.supported_formats)
    for format, format_info in converter.all_formats().items():
        format_info = deepcopy(format_info)
        format_info['name'] = format.name
        if format.name not in supported_formats:
            format_info['disabled'] = True
        formats.append(format_info)
    return sorted(formats, key=lambda f: f.get('disabled', False))

In [ ]:
get_export_formats(editor_config)

In [8]:
class Format(Enum):
    JSON = 1
    JSON_MIN = 2
    CSV = 3
    TSV = 4
    CONLL2003 = 5
    COCO = 6
    VOC = 7
    BRUSH_TO_NUMPY = 8
    BRUSH_TO_PNG = 9
    ASR_MANIFEST = 10
    YOLO = 11

    def __str__(self):
        return self.name

    @classmethod
    def from_string(cls, s):
        try:
            return Format[s]
        except KeyError:
            raise ValueError()

In [44]:
format=Format.from_string("COCO")
format == Format.COCO

True

In [41]:
from tempfile import mkdtemp
mkdtemp()

'/tmp/tmpirfokajv'

In [51]:
project_dir='/home/rchuzh/.local/share/label-studio/media/upload'
task_json="""{
        "id": 2,
        "annotations": [
            {
                "id": 13,
                "completed_by": {
                    "id": 3,
                    "email": "ecyzc@nottingham.edu.my",
                    "first_name": "",
                    "last_name": ""
                },
                "result": [
                    {
                        "original_width": 3000,
                        "original_height": 4000,
                        "image_rotation": 0,
                        "value": {
                            "x": 17.600000000000005,
                            "y": 47.1,
                            "width": 32,
                            "height": 23.5,
                            "rotation": 0,
                            "rectanglelabels": [
                                "Hello"
                            ]
                        },
                        "id": "M4mae6iYcI",
                        "from_name": "tag",
                        "to_name": "img",
                        "type": "rectanglelabels"
                    },
                    {
                        "original_width": 3000,
                        "original_height": 4000,
                        "image_rotation": 0,
                        "value": {
                            "x": 55.86666666666667,
                            "y": 27.60000000000001,
                            "width": 15.333333333333334,
                            "height": 11.7,
                            "rotation": 0,
                            "rectanglelabels": [
                                "Hello"
                            ]
                        },
                        "id": "76y19AdsUB",
                        "from_name": "tag",
                        "to_name": "img",
                        "type": "rectanglelabels"
                    },
                    {
                        "original_width": 3000,
                        "original_height": 4000,
                        "image_rotation": 0,
                        "value": {
                            "x": 68,
                            "y": 42.3,
                            "width": 15.333333333333334,
                            "height": 12.5,
                            "rotation": 0,
                            "rectanglelabels": [
                                "Hello"
                            ]
                        },
                        "id": "Dh9hUuSc8Z",
                        "from_name": "tag",
                        "to_name": "img",
                        "type": "rectanglelabels"
                    }
                ],
                "was_cancelled": false,
                "ground_truth": false,
                "created_at": "2021-06-14T09:35:28.773001Z",
                "updated_at": "2021-06-18T07:52:24.910539Z",
                "lead_time": 17023.217,
                "prediction": {},
                "result_count": 0,
                "task": 2
            }
        ],
        "predictions": [],
        "file_upload": "IMG_20210315_181133.jpg",
        "data": {

            "image": "/home/rchuzh/Documents/aruco/train/IMG_20210315_181133.jpg"
        },
        "meta": {},
        "created_at": "2021-06-14T08:44:41.014786Z",
        "updated_at": "2021-06-18T07:52:24.858387Z",
        "project": 8
    }"""

In [54]:
tasks=json.loads(task_json)
type(tasks)

dict

In [38]:
import hashlib
data = json.dumps(tasks, ensure_ascii=False)
md5 = hashlib.md5(json.dumps(data).encode('utf-8')).hexdigest()

In [47]:

converter=Converter(editor_config2,project_dir=project_dir)

In [48]:
display(vars(converter))

{'project_dir': '/home/rchuzh/.local/share/label-studio/media/upload',
 'upload_dir': None,
 '_schema': {'label': {'type': 'RectangleLabels',
   'to_name': ['image'],
   'inputs': [{'type': 'Image', 'value': 'image'}],
   'labels': ['Hello', 'World'],
   'labels_attrs': {'Hello': {'value': 'Hello', 'background': 'green'},
    'World': {'value': 'World', 'background': 'blue'}}},
  'choice': {'type': 'Choices',
   'to_name': ['image'],
   'inputs': [{'type': 'Image', 'value': 'image'}],
   'labels': ['Boeing', 'Airbus'],
   'labels_attrs': {'Boeing': {'value': 'Boeing', 'background': 'blue'},
    'Airbus': {'value': 'Airbus', 'background': 'green'}}}},
 '_data_keys': ['image'],
 '_output_tags': ['label', 'choice'],
 '_supported_formats': ['JSON',
  'JSON_MIN',
  'CSV',
  'TSV',
  'COCO',
  'VOC',
  'YOLO']}

In [55]:
item_iterator=converter.iter_from_json_file(tasks,is_string=True)
random_list=[]
for item_idx, item in enumerate(item_iterator):
    random_list.append(item)

Enter string
Done


In [65]:
random_list=[]
for item_idx, item in enumerate(item_iterator):
    random_list.append(item)


In [56]:
random_list

[]

In [55]:
class random:
    def __init__ (self,age,name):
        self.age=age
        self.name=name
    
    def test1(self,some_text,some_flag=False):
        def innertest1(data,age):
            print(data,age)

        if some_flag:
            # innertest1(some_text,self.age)
            print(self.age)

        else:
            innertest1("else block",self.age)



In [57]:
randomtest=random(22,'Alex')
randomtest.test1("Hello",True)

22


In [47]:
def nextSquare():
    i = 1
  
    # An Infinite loop to generate squares 
    while True:
        yield i*i                
        i += 1  # Next execution resumes 
                # from this point    


In [48]:
for num in nextSquare():
    if num > 100:
         break    
    print(num)


1
4
9
16
25
36
49
64
81
100
